In [1]:
from LLMPromptGenerator import LLMPromptGenerator
from GenMusicFromPrompt import GenMusicFromPrompt

from audiocraft.models import MusicGen
from audiocraft.models import MultiBandDiffusion
from audiocraft.utils.notebook import display_audio

import math
import torchaudio
import torch
from audiocraft.utils.notebook import display_audio
from tqdm import tqdm
import json
import random
import os

/home/grantsl/anaconda3/envs/pages/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.1.0+cu121 with CUDA 1201 (you have 2.1.0+cu118)
    Python  3.10.13 (you have 3.10.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


In [2]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

GROUP_WORD_COUNT = 60 #120 
SONG_DUR_SECONDS = 30 #60 
PREV_SONG_DUR = 2 # 4

MAX_GROUP_CNT = 3 # limits the number of groups to be processing

cuda


In [3]:
extractor = LLMPromptGenerator() # device=device


In [4]:
def load_word_sections(file_path, desired_section_size):
    with open(file_path) as f: 
        book_text = f.read()

    words = book_text.split()

    # Calculate the total number of words
    total_words = len(words)

    # Determine the number of sections, aiming for equally sized sections
    # Calculate the optimal number of sections to avoid a significantly shorter final section
    optimal_num_sections = round(total_words / desired_section_size)

    # Calculate the new section size to more evenly distribute words across sections
    new_section_size = total_words // optimal_num_sections if total_words % optimal_num_sections == 0 else (total_words // optimal_num_sections) + 1

    # Adjust the last section to avoid being too short
    if total_words % new_section_size < new_section_size / 2:
        optimal_num_sections += 1

    word_sections = [' '.join(words[i:i+new_section_size]) for i in range(0, total_words, new_section_size)]
    return word_sections

In [6]:
# book_text_loc = 'prompt_examples/fotr_gandalf_balrog.txt'
# book_text_loc = 'prompt_examples/fotr_gandalf_balrog.txt'


def full_pipeline(book_text_loc,
                  extractor=None,
                  generator=None,
                  group_word_count=GROUP_WORD_COUNT, 
                  max_group_count=MAX_GROUP_CNT,
                  song_dur_seconds=SONG_DUR_SECONDS, 
                  previous_song_duration=PREV_SONG_DUR,
                  device="cpu",
                  save_file_loc=None,
                  verbose=True,
                  flush_extractor=True):
    # Load in Text and Split into Sections
    word_sections = load_word_sections(book_text_loc, group_word_count)

    if max_group_count is not None and len(word_sections) > max_group_count:
        word_sections = word_sections[:max_group_count]

    if verbose:
        print('section count:', len(word_sections))

    if extractor is None:
        if verbose:
            print('Creating new extractor')
        extractor = LLMPromptGenerator()
    
    # Extract JSON Info
    extracted_json = extractor.extract_info(word_sections, flush=flush_extractor)
    # Generate Prompts
    prompts = extractor.prompts()
    
    print(extractor.info)
   
    if verbose:
        print('Prompts:')
        for p in prompts:
            print(p)
    
    if generator is None:
        if verbose:
            print('Creating new generator')
        generator = GenMusicFromPrompt(duration=song_dur_seconds, device=device, previous_song_duration=previous_song_duration)
    

    music = generator.generate_from_list(prompts, verbose=verbose, flush=True)
    if save_file_loc is not None:
        if os.path.isdir(save_file_loc):
            base_name = os.path.basename(book_text_loc).split('.')[0] + '.wav'
            save_file_loc = os.path.join(save_file_loc, base_name)
        generator.save_audio(save_file_loc)
    
    return generator, prompts #, extracted_json



In [7]:
# book_text_loc = 'prompt_examples/wok_war.txt'
book_text_loc = 'prompt_examples/fotr_gandalf_balrog.txt'

# book_text_loc = 'prompt_examples/cith_things.txt'
# book_text_loc = './prompt_examples/rots_obiewan_vs_anakin_pt1.txt'
# book_text_loc = './prompt_examples/raj_romeo_stalking_pt1.txt'


output_dir = './generated_examples_midterm/'
print('book_text_loc:', book_text_loc)
generator, prompts = full_pipeline(book_text_loc, extractor=extractor, save_file_loc=output_dir, device=device)

book_text_loc: prompt_examples/fotr_gandalf_balrog.txt
section count: 3


/home/grantsl/anaconda3/envs/pages/lib/python3.10/site-packages/transformers/generation/utils.py:1295: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


[MusicGenInfo(short_term=ShortTermAttributes(tone='mystical', intensity='soft', is_crescendo=False, volume='low'), long_term=LongTermAttributes(instrumentation='orchestral', short_background_ambient_setting='mystical forest', short_music_descriptors='ethereal', pitch='C', beat='slow', is_major_key=True)), MusicGenInfo(short_term=ShortTermAttributes(tone='mellow', intensity='soft and subtle', is_crescendo=False, volume='low'), long_term=LongTermAttributes(instrumentation='piano', short_background_ambient_setting='a soft, gentle melody', short_music_descriptors='a soothing, calming atmosphere', pitch='C', beat='slow and steady', is_major_key=True)), MusicGenInfo(short_term=ShortTermAttributes(tone='mysterious', intensity='soft', is_crescendo=False, volume='low'), long_term=LongTermAttributes(instrumentation='orchestral', short_background_ambient_setting='mysterious', short_music_descriptors='ethereal', pitch='middle', beat=' steady', is_major_key=False))]
Prompts:
mystical forest ambient

/home/grantsl/anaconda3/envs/pages/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
  0%|          | 0/3 [00:00<?, ?it/s]

/home/grantsl/anaconda3/envs/pages/lib/python3.10/site-packages/torch/nn/modules/conv.py:306: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv1d(input, weight, bias, self.stride,
 33%|███▎      | 1/3 [01:26<02:53, 86.79s/it]

 67%|██████▋   | 2/3 [02:51<01:25, 85.27s/it]

100%|██████████| 3/3 [04:15<00:00, 85.08s/it]

In [ ]:
for prompt in prompts:
    print(prompt)
    print('---')

outdoors setting, calm tone, low intensity, piano instrumentals, C pitch
---
A dark and stormy night setting, Serene tone, Soft intensity, Piano instrumentals, Middle C pitch
---
a clear summer evening setting, serene tone, soft intensity, piano instrumentals, C major pitch
---


## Left to work on

#### Before Midterm
* Metric for generated music - What is good?
* Look at generation times vs quality (Facebook model - Is live generation feasible?)

#### After Midterm
* Modify the LLM class to pass in a giant text file, batch, and generate json
* Pipeline the LLM and GenMusic class in a pipeline class
* Prompt engineering
* Fun: Karaoke interface 
